In [1]:
import pickle

In [2]:
with open('mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)

In [4]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [5]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [6]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [14]:
train_images=[]
train_labels=[]
path = "./PennFudanPed/PennFudanPed/PNGImages/"
for e,i in enumerate(mapping.keys()):
    filename = i+".png"
    print(e,filename)
    image = cv2.imread(os.path.join(path,filename))
    gtvalues=[]
    for pt in mapping[i]:
        x1 = pt[0]
        y1 = pt[1]
        x2 = pt[2]
        y2 = pt[3]
        gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = image.copy()
    counter = 0
    falsecounter = 0
    flag = 0
    fflag = 0
    bflag = 0
    for e,result in enumerate(ssresults):
        if e < 2000 and flag == 0:
            for gtval in gtvalues:
                x,y,w,h = result
                iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                if counter < 30:
                    if iou > 0.70:
                        timage = imout[y:y+h,x:x+w]
                        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(1)
                        counter += 1
                else :
                    fflag =1
                if falsecounter <30:
                    if iou < 0.3:
                        timage = imout[y:y+h,x:x+w]
                        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(0)
                        falsecounter += 1
                else :
                    bflag = 1
            if fflag == 1 and bflag == 1:
                print("inside")
                flag = 1

0 FudanPed00001.png
1 FudanPed00002.png
2 FudanPed00003.png
3 FudanPed00004.png
4 FudanPed00005.png
5 FudanPed00006.png
inside
6 FudanPed00007.png
7 FudanPed00008.png
8 FudanPed00009.png
9 FudanPed00010.png
10 FudanPed00011.png
11 FudanPed00012.png
12 FudanPed00013.png
13 FudanPed00014.png
14 FudanPed00015.png
15 FudanPed00016.png
16 FudanPed00017.png
17 FudanPed00018.png
18 FudanPed00019.png
19 FudanPed00020.png
20 FudanPed00021.png
21 FudanPed00022.png
22 FudanPed00023.png
23 FudanPed00024.png
24 FudanPed00025.png
25 FudanPed00026.png
26 FudanPed00027.png
27 FudanPed00028.png
28 FudanPed00029.png
29 FudanPed00030.png
30 FudanPed00031.png
31 FudanPed00032.png
32 FudanPed00033.png
33 FudanPed00034.png
34 FudanPed00035.png
35 FudanPed00036.png
36 FudanPed00037.png
37 FudanPed00038.png
38 FudanPed00039.png
39 FudanPed00040.png
40 FudanPed00041.png
41 FudanPed00042.png
42 FudanPed00043.png
43 FudanPed00044.png
44 FudanPed00045.png
45 FudanPed00046.png
46 FudanPed00047.png
47 FudanPed00048

In [15]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [17]:
with open('x_new.pkl', 'wb') as f:
    pickle.dump(X_new, f)

In [19]:
with open('y_new.pkl', 'wb') as f:
    pickle.dump(y_new, f)